# Train a Xgboost Model with Watson Machine Learning 

Notebook created by Zeming Zhao on June, 2021

XGBoost is an implementation of gradient boosted decision trees designed for speed and performance. which is an algorithm that has recently been dominating applied machine learning and Kaggle competitions for structured or tabular data.

SnapBoost implements a boosting machine that can be used to construct an ensemble of decision trees. It can be used for both clasification and regression tasks. In constrast to other boosting frameworks, Snap ML’s boosting machine dose not utilize a fixed maximal tree depth at each boosting iteration. Instead, the tree depth is sampled at each boosting iteration according to a discrete uniform distribution. The fit and predict functions accept numpy.ndarray data structures.

This notebook covers the following sections:

1. [Setup Xgboost Model using xgboost lib](#xgboost-model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#xgboost-cpu)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#xgboost-gpu)<br>

1. [Setup SnapBoost Model using snapML lib on CPU](#snapml-model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#snapml-cpu)<br>

1. [Setup SnapBoost Model using snapML lib on GPU](#snapml-model-gpu)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#snapml-gpu)<br>

## Preparations
### Prepare directory and file for writing Xgboost engine.

In [11]:
from pathlib import Path
model_dir = f'/project_data/data_asset/models' 
model_base_name = f'main.py'
Path(model_dir).mkdir(exist_ok=True)
print("create model directory done.")

create model directory done.


<a id = "xgboost-model"></a>
## Step 1 : Setup Xgboost model
### create a Xgboost Model based on Xgboost lib 

In [12]:
model_main='xgboost-'+model_base_name

In [13]:
%%writefile {model_dir}/{model_main}

import os, datetime
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
import xgboost as xgb

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Define Parameters for a large regression
n_samples = 2**13 
n_features = 899 
n_info = 600 
data_type = np.float32

# Generate Data using scikit-learn
X,y = make_classification(n_samples=n_samples,
                          n_features=n_features,
                          n_informative=n_info,
                          random_state=123, n_classes=2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y.astype(np.int32))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state=0)

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

# set parameters
param = {
    'eta': 0.3, 
    'max_depth': 6,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

# setup model and train
start = datetime.datetime.now()
model = xgb.train(param, D_train, steps)
end = datetime.datetime.now()
print ("Xgboost train timecost: %.2gs" % ((end-start).total_seconds()))

# predict
start = datetime.datetime.now()
preds = model.predict(D_test)
end = datetime.datetime.now()
print ("Xgboost predict timecost: %.2gs" % ((end-start).total_seconds()))

# check result
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

# save the xgboost model into a file
import pickle
filename = './xgboost_model.pkl'
pickle.dump(model, open(filename, 'wb'))              
print("Xgboost model saved successfully.")

Overwriting /project_data/data_asset/models/xgboost-main.py


<a id = "xgboost-cpu"></a>
## Step 2 :  Training the Xgboost model on CPU with Watson Machine Learning Accelerator
### Prepare the libs for job submission

In [4]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib

Populating the interactive namespace from numpy and matplotlib


### Configuring your environment and project details
To set up your project details, provide your credentials in this cell. You must include your cluster URL, username, and password.

In [5]:
# please enter Watson Machine Learning Accelerator host name
hostname='wmla-console-wmla.apps.dse-perf.cpolab.ibm.com'
# login='username:password' # please enter the login and password
login='mluser1:mluser1'
es = base64.b64encode(login.encode('utf-8')).decode("utf-8")
# print(es)
commonHeaders={'Authorization': 'Basic '+es}
req = requests.Session()
auth_url = 'https://{}/auth/v1/logon'.format(hostname)
print(auth_url)
a=requests.get(auth_url,headers=commonHeaders, verify=False)
access_token=a.json()['accessToken']
# print("Access_token: ", access_token)
dl_rest_url = 'https://{}/platform/rest/deeplearning/v1'.format(hostname)
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()
# Health check
confUrl = 'https://{}/platform/rest/deeplearning/v1/conf'.format(hostname)
r = req.get(confUrl, headers=commonHeaders, verify=False)

https://wmla-console-wmla.apps.dse-perf.cpolab.ibm.com/auth/v1/logon


### Define the status checking fuction

In [6]:
import tarfile
import tempfile
import os
import json
import pprint
import pandas as pd
from IPython.display import clear_output

def query_job_status(job_id,refresh_rate=3) :

    execURL = dl_rest_url  +'/execs/'+ job_id['id']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(execURL, headers=commonHeaders, verify=False)
        monitoring = pd.DataFrame(res.json(), index=[0])
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(monitoring)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['PENDING_CRD_SCHEDULER', 'SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res

### Define the job submission fuction

In [14]:
def submit_job_to_wmla (args, files) :
    starttime = datetime.datetime.now()
    r = requests.post(dl_rest_url+'/execs?args='+args, files=files,
                  headers=commonHeaders, verify=False)
    if not r.ok:
        print('submit job failed: code=%s, %s'%(r.status_code, r.content))
    job_status = query_job_status(r.json(),refresh_rate=5)
    endtime = datetime.datetime.now()
    print("\nTotallly training cost: ", (endtime - starttime).seconds, " seconds.")

### Define the submittion parameters

In [15]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType cpu \
--conda-env-name rapids-21.06-new \
--model-main ' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --conda-env-name rapids-21.06-new --model-main xgboost-main.py


In [16]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-858,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --...,wmla-858,mluser1,FINISHED,wmla-858,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-858/_submitted_code,SingleNodeTensorflowTrain,2021-07-27T08:41:21Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-858',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu '
          '--conda-env-name rapids-21.06-new --model-main xgboost-main.py ',
  'createTime': '2021-07-27T08:41:21Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-858',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-858',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-858/_submitted_code'}

Totallly training cost:  2840  seconds.


<a id = "xgboost-gpu"></a>
## Step 3 :  Training the Xgboost model on GPU with Watson Machine Learning Accelerator
### Define the submittion parameters using conda env with GPU version XGBoost

In [29]:
# specify the conda env of xgboost and worker device type
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType gpu \
--conda-env-name dlipy3  \
--model-main ' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --conda-env-name dlipy3  --model-main xgboost-main.py


In [30]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-860,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --...,wmla-860,mluser1,FINISHED,wmla-860,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-860/_submitted_code,SingleNodeTensorflowTrain,2021-07-27T09:47:24Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-860',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu '
          '--conda-env-name dlipy3  --model-main xgboost-main.py ',
  'createTime': '2021-07-27T09:47:24Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-860',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-860',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-860/_submitted_code'}

Totallly training cost:  62  seconds.


<a id = "snapml-model"></a>
## Step 4 :  Setup Snap Boosting model using snapML
### Create a Snap Boosting model based on snapML 

In [31]:
model_main='snapboost-'+model_base_name

In [39]:
%%writefile {model_dir}/{model_main}

import os, datetime
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from snapml import BoostingMachineClassifier as SnapBoostingMachineClassifier

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Define Parameters for a large regression
n_samples = 2**13 
n_features = 899 
n_info = 600 
data_type = np.float32

# Generate Data using scikit-learn
X,y = make_classification(n_samples=n_samples,
                          n_features=n_features,
                          n_informative=n_info,
                          random_state=123, n_classes=2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y.astype(np.int32))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state=0)

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

snap_model = SnapBoostingMachineClassifier(random_state=42, 
                                    n_jobs=1,
                                    hist_nbins=256,
                                    num_round=20)  # same interations number with xgboost

# setup model and train
start = datetime.datetime.now()
snap_model.fit(X_train.values, y_train.values)  #.train(param, D_train, steps)
end = datetime.datetime.now()
print ("Xgboost train timecost: %.2gs" % ((end-start).total_seconds()))

# predict
start = datetime.datetime.now()
preds = snap_model.predict(X_test.values)
end = datetime.datetime.now()
print ("Xgboost predict timecost: %.2gs" % ((end-start).total_seconds()))

# check result
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

# save the xgboost model into a file
import pickle
filename = './snapboost_model.pkl'
pickle.dump(snap_model, open(filename, 'wb'))              
print("Snapboost model saved successfully.")

Overwriting /project_data/data_asset/models/snapboost-gpu-main.py


<a id = "snapml-cpu"></a>
## Step 5 :  Training the SnapML model on CPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [40]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType cpu \
--conda-env-name snapml-177rc \
--model-main ' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --conda-env-name snapml-177rc --model-main snapboost-gpu-main.py


In [41]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-863,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --...,wmla-863,mluser1,FINISHED,wmla-863,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-863/_submitted_code,SingleNodeTensorflowTrain,2021-07-27T09:58:06Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-863',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu '
          '--conda-env-name snapml-177rc --model-main snapboost-gpu-main.py ',
  'createTime': '2021-07-27T09:58:06Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-863',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-863',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-863/_submitted_code'}

Totallly training cost:  61  seconds.


<a id = "snapml-model-gpu"></a>
## Step 6 :  Setup SnapBoost model using snapML on GPU
### Create a SnapBoost model based on snapML 

In [35]:
model_main='snapboost-gpu-'+model_base_name

In [36]:
%%writefile {model_dir}/{model_main}

import os, datetime
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from snapml import BoostingMachineClassifier as SnapBoostingMachineClassifier

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Define Parameters for a large regression
n_samples = 2**13 
n_features = 899 
n_info = 600 
data_type = np.float32

# Generate Data using scikit-learn
X,y = make_classification(n_samples=n_samples,
                          n_features=n_features,
                          n_informative=n_info,
                          random_state=123, n_classes=2)

X = pd.DataFrame(X.astype(data_type))
y = pd.Series(y.astype(np.int32))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state=0)

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

snap_model = SnapBoostingMachineClassifier(use_gpu=True,   # use gpu
                                    random_state=42, 
                                    n_jobs=1,
                                    hist_nbins=256,
                                    num_round=20)  # same interations number with xgboost

# setup model and train
start = datetime.datetime.now()
snap_model.fit(X_train.values, y_train.values)  #.train(param, D_train, steps)
end = datetime.datetime.now()
print ("Xgboost train timecost: %.2gs" % ((end-start).total_seconds()))

# predict
start = datetime.datetime.now()
preds = snap_model.predict(X_test.values)
end = datetime.datetime.now()
print ("Xgboost predict timecost: %.2gs" % ((end-start).total_seconds()))

# check result
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

# save the xgboost model into a file
import pickle
filename = './snapboost_model.pkl'
pickle.dump(snap_model, open(filename, 'wb'))              
print("Snapboost model saved successfully.")

Overwriting /project_data/data_asset/models/snapboost-gpu-main.py


<a id = "snapml-gpu"></a>
## Step 7 :  Training the SnapML model on GPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [37]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType gpu \
--conda-env-name snapml-177rc \
--model-main ' + model_main
# --msd-env CUDA_FORCE_PTX_JIT=1 \
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --conda-env-name snapml-177rc --model-main snapboost-gpu-main.py


In [38]:
#files = {'file': open('/project_data/data_asset/models/snapboost-gpu-main.py', 'rb')}
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-862,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --...,wmla-862,mluser1,FINISHED,wmla-862,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-862/_submitted_code,SingleNodeTensorflowTrain,2021-07-27T09:55:56Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-862',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu '
          '--conda-env-name snapml-177rc --model-main snapboost-gpu-main.py ',
  'createTime': '2021-07-27T09:55:56Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-862',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-862',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-862/_submitted_code'}

Totallly training cost:  51  seconds.
